Importing all necessary libraries

In [5]:
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import seaborn as sns

'Certified Fresh' , Genre :'Adventure_movies' : Rating : 'G'

In [6]:

movie_dict={'Title':[],'Rating':[],'Genre':[], "Release Date":[], "Reviews Count":[],"Rating Code":[],"Duration":[]}
movie_links=[]
BASE_URL="https://www.rottentomatoes.com"
def fetchUrl(url):
    try:
        response = requests.get(url)
        soup=BeautifulSoup(response.text, 'html.parser')
        return soup
    except Exception as e:
        print("Error: ",e)
def get_movie_links(href):
        movie_links.append(BASE_URL + href)
        return movie_links
def scrapingData(url):
    soup=fetchUrl(url)
    movieTitle=soup.find("h1",attrs={"slot":"titleIntro"})
    movieRatingsCode=soup.find("rt-text",attrs={"slot":"ratingsCode"})
    movieReleaseDate=soup.find("rt-text",attrs={"slot":"releaseDate"})
    movieGenre=soup.find("rt-text",attrs={"slot":"genre"})
    movieDuration=soup.find("rt-text",attrs={"slot":"duration"})
    movieRating=soup.find("rt-button",attrs={"slot":"criticsScore"})
    movieReviewsCount=soup.find("rt-link",attrs={"slot":"criticsReviews"})
    movie_dict["Title"].append(movieTitle.text.strip())
    if movieRating:
        movie_dict["Rating"].append(movieRating.text.strip())
    if movieGenre:
        movie_dict["Genre"].append(movieGenre.text.strip())
    if movieReleaseDate:
        movie_dict["Release Date"].append(movieReleaseDate.text.strip())
    if movieRatingsCode:
        movie_dict["Rating Code"].append(movieRatingsCode.text.strip())
    if movieDuration:
        movie_dict["Duration"].append(movieDuration.text.strip())
    if movieReviewsCount:
        movie_dict["Reviews Count"].append(movieReviewsCount.text.strip())
    return movie_dict
soup=fetchUrl(BASE_URL)
totalLinks=soup.find_all('a',attrs={'data-track':"scores" , "slot":"caption"})
totalLinks = [tag.get('href') for tag in totalLinks]
for href in totalLinks:
    movieUrl=get_movie_links(href)

count=0
for movies in movieUrl:
    movieData=scrapingData(movies)
print(movieData)


{'Title': ['Twisters', 'Despicable Me 4', 'Paris Olympics on NBC at AMC Theatres', "NBC's Paris Olympics Opening Ceremony in IMAX", 'Longlegs', 'Fly Me to the Moon', 'Deadpool & Wolverine', 'MaXXXine', 'Touch', 'A Quiet Place: Day One', 'Inside Out 2', 'Sing Sing', 'Lumina', 'Princess Mononoke', 'Horizon: An American Saga - Chapter 1', 'Kinds of Kindness', 'Thelma', 'Season 1 – Those About to Die', 'Season 6 – Cobra Kai', 'Season 1 – Lady in the Lake', 'Season 4 – Snowpiercer', 'Season 1 – Star Wars: The Acolyte', 'Season 1 – Sunny', 'Season 2 – House of the Dragon', 'Thelma', 'Supacell', 'Season 3 – Vikings: Valhalla', 'Season 1 – Sausage Party: Foodtopia', 'My Spy the Eternal City', 'The Imaginary', 'Season 4 – The Boys', 'Young Woman and the Sea', 'Season 3 – The Bear', "Season 2 – Marvel's Hit-Monkey", 'Season 1 – My Lady Jane', 'Beverly Hills Cop: Axel F', 'Season 1 – Land of Women', 'Despicable Me 4', 'Longlegs', 'Inside Out 2', 'A Quiet Place: Day One', 'Fly Me to the Moon', 'Ba

In [7]:
print(len(movieData["Title"]),len(movieData["Rating"]),len(movieData["Genre"]),len(movieData["Release Date"]),len(movieData["Rating Code"]),len(movieData["Duration"]),len(movieData["Reviews Count"]))
if len(movieData["Rating Code"]) < 94:
    movieData["Rating Code"].extend([None] * (len(movieData['Title']) - len(movieData["Rating Code"])))

if len(movieData["Duration"]) < 94:
    movieData["Duration"].extend([None] * (len(movieData['Title']) - len(movieData["Duration"])))

for key in movieData.keys():
    print(f"{key}: {len(movieData[key])}")


91 91 91 91 58 71 91
Title: 91
Rating: 91
Genre: 91
Release Date: 91
Reviews Count: 91
Rating Code: 91
Duration: 91


create a dataframe and store a csv file for all scrape data


In [8]:
df=pd.DataFrame.from_dict(movieData)
df.to_csv("movie_data.csv",index=False)

In [9]:
from openpyxl import Workbook
import numpy as np

wb=Workbook()
ws=wb.active
ws.title="Movie Data"
a=['Twisters',
'Despicable Me 4',
'Paris Olympics on NBC at AMC Theatres',
"NBC's Paris Olympics Opening Ceremony in IMAX",
'Fly Me to the Moon',
'MaXXXine',
'Longlegs',
'Touch'
]
b=['PG-13',
'PG',
'PG-13'
]
temp_dict={"title":a,"rating":b}

for key in temp_dict.keys():
    ws.append([key]+temp_dict[key])
np.transpose(temp_dict['title'])
np.transpose(temp_dict['rating'])
wb.save("movie_data.xlsx")

In [10]:
df.head(5)


,Title,Rating,Genre,Release Date,Reviews Count,Rating Code,Duration
0,Twisters,77%,Action,In Theaters Fri Jul 19,91 Reviews,PG-13,2h 2m
1,Despicable Me 4,55%,Kids & Family,Now Playing,148 Reviews,PG,1h 35m
2,Paris Olympics on NBC at AMC Theatres,,Sports,In Theaters Fri Jul 26,Reviews,R,4h 0m
3,NBC's Paris Olympics Opening Ceremony in IMAX,,Sports,In Theaters Fri Jul 26,Reviews,PG-13,1h 41m
4,Longlegs,87%,Horror,Now Playing,202 Reviews,R,2h 12m
